In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd

In [ ]:
from huggingface_hub import login

# Log in to Hugging Face Hub using authentication token
# Required for accessing private models and pushing models to Hub
login("token")

In [ ]:
!pip install datasets

   ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 480.6/480.6 kB 10.8 MB/s eta 0:00:00
   ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 179.3/179.3 kB 17.5 MB/s eta 0:00:00
   ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 194.1/194.1 kB 17.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolve

In [ ]:
from datasets import load_dataset
import pandas as pd

# Load sentiment analysis dataset from Hugging Face Hub
# Dataset contains 100k Weibo posts with sentiment labels
ds = load_dataset("dirtycomputer/weibo_senti_100k")
ds

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


weibo_senti_100k.csv:   0%|          | 0.00/19.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/119988 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'review'],
        num_rows: 119988
    })
})

In [ ]:
# Convert Hugging Face dataset to pandas DataFrame
df = pd.DataFrame(ds['train'])

# Display basic information about the DataFrame
print("Dataset Overview:")
print(f"Number of samples: {len(df)}")
print(f"Columns: {df.columns.tolist()}")
print("\nLabel distribution:")
print(df['label'].value_counts())
print("\nSample reviews:")
print(df['review'].head())

# Check for any missing values
if df.isnull().sum().any():
    print("\nWarning: Dataset contains missing values!")

Dataset Overview:
Number of samples: 119988
Columns: ['label', 'review']

Label distribution:
label
0    59995
1    59993
Name: count, dtype: int64

Sample reviews:
0                ÔªøÊõ¥Âçö‰∫ÜÔºåÁàÜÁÖß‰∫ÜÔºåÂ∏ÖÁöÑÂëÄÔºåÂ∞±ÊòØË∂äÊù•Ë∂äÁà±‰Ω†ÔºÅÁîüÂø´ÂÇªÁº∫[Áà±‰Ω†][Áà±‰Ω†][Áà±‰Ω†]
1    @Âº†ÊôìÈπèjonathan ÂúüËÄ≥ÂÖ∂ÁöÑ‰∫ãË¶ÅËÆ§ÁúüÂØπÂæÖ[ÂìàÂìà]ÔºåÂê¶ÂàôÁõ¥Êé•ÂºÄÈô§„ÄÇ@‰∏Å‰∏ÅÁúã‰∏ñÁïå ÂæàÊòØÁªÜÂøÉ...
2    ÂßëÂ®òÈÉΩÁæ°ÊÖï‰Ω†Âë¢‚Ä¶ËøòÊúâÊãõË¥¢Áå´È´òÂÖ¥‚Ä¶‚Ä¶//@Áà±Âú®ËîìÂª∂-JC:[ÂìàÂìà]Â∞èÂ≠¶Âæí‰∏ÄÊûöÔºåÁ≠âÁùÄÊòéÂ§©ËßÅÊÇ®Âë¢/...
3                                           Áæé~~~~~[Áà±‰Ω†]
4                                    Ê¢¶ÊÉ≥ÊúâÂ§öÂ§ßÔºåËàûÂè∞Â∞±ÊúâÂ§öÂ§ß![ÈºìÊéå]
Name: review, dtype: object


In [ ]:
from sklearn.model_selection import train_test_split

# Split dataset into training and test sets
# - test_size=0.2: 80% training, 20% testing
# - shuffle=True: randomly shuffle before splitting
# - random_state=42: set seed for reproducibility
train_df, test_df = train_test_split(df, test_size=0.2, shuffle=True, random_state=42)

In [ ]:
# Check for available CUDA device and set up GPU/CPU
# Colab typically provides a single GPU, if available
if torch.cuda.is_available():
   device = torch.device("cuda")
   # Print GPU information
   print(f"Using GPU: {torch.cuda.get_device_name(0)}")
   print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
   device = torch.device("cpu")
   print("No GPU available, using CPU")

Using GPU: NVIDIA A100-SXM4-40GB
GPU Memory: 42.48 GB


In [ ]:
# Initialize the BERT Chinese tokenizer
# Uses bert-base-chinese pre-trained model's vocabulary
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

In [ ]:
from torch.utils.data import Dataset
import torch
class TextDataset(Dataset):
   """
   Custom Dataset class for text data, inheriting from PyTorch's Dataset.

   Parameters:
   tokenizer (Tokenizer): Tokenizer object for text encoding
   texts (list): List of text samples
   labels (list): List of corresponding labels
   """
   def __init__(self, tokenizer, texts, labels):
       # Encode texts with padding and truncation
       self.encodings = tokenizer(
           texts,
           truncation=True,
           padding=True,
           max_length=512,  # Explicitly set max length for BERT
           return_tensors='pt'  # Return PyTorch tensors directly
       )
       # Convert labels to tensor
       self.labels = torch.tensor(labels)

   def __getitem__(self, idx):
       """
       Get a single sample by index.

       Args:
           idx (int): Sample index

       Returns:
           dict: Dictionary containing encoded text data and label
       """
       return {
           'input_ids': self.encodings['input_ids'][idx],
           'attention_mask': self.encodings['attention_mask'][idx],
           'labels': self.labels[idx]
       }

   def __len__(self):
       """
       Get dataset length.

       Returns:
           int: Number of samples in dataset
       """
       return len(self.labels)

In [ ]:
# Print unique labels in the dataset
print("Unique labels in training data:", sorted(set(train_df['label'])))

# Create explicit label mapping
label_mapping = {
    0: 0,  # Negative
    1: 1   # Positive
}

# Map labels using explicit mapping
train_labels = [label_mapping[label] for label in train_df['label']]
test_labels = [label_mapping[label] for label in test_df['label']]

# Verify label distribution after mapping
print("\nLabel distribution after mapping:")
print("Training:", pd.Series(train_labels).value_counts())
print("Testing:", pd.Series(test_labels).value_counts())

Unique labels in training data: [0, 1]

Label distribution after mapping:
Training: 0    48151
1    47839
Name: count, dtype: int64
Testing: 1    12154
0    11844
Name: count, dtype: int64


In [ ]:
# Explicit label mapping to ensure correct sentiment assignment
label_to_index = {
    0: 0,  # Keep negative as 0
    1: 1   # Keep positive as 1
}

# Map labels using explicit mapping
train_labels = [label_to_index[label] for label in train_df['label']]
test_labels = [label_to_index[label] for label in test_df['label']]

# Create datasets with verified labels
train_dataset = TextDataset(tokenizer, train_df['review'].tolist(), train_labels)
test_dataset = TextDataset(tokenizer, test_df['review'].tolist(), test_labels)

# Verify final mapping
print("\nFinal verification:")
print("Training set label distribution:", pd.Series(train_labels).value_counts())
print("Test set label distribution:", pd.Series(test_labels).value_counts())


Final verification:
Training set label distribution: 0    48151
1    47839
Name: count, dtype: int64
Test set label distribution: 1    12154
0    11844
Name: count, dtype: int64


In [ ]:
# Load pre-trained Chinese BERT model and configure for binary classification
model = BertForSequenceClassification.from_pretrained(
   'bert-base-chinese',
   num_labels=2  # Binary classification (negative/positive)
)
model = model.to(device)

# Define training arguments optimized for A100 GPU
training_args = TrainingArguments(
   output_dir='sentiment-weibo-100k-fine-tuned-bert',
   num_train_epochs=3,
   per_device_train_batch_size=32,  # Increased for A100 GPU
   per_device_eval_batch_size=64,   # Increased for evaluation
   warmup_steps=500,
   weight_decay=0.01,
   logging_dir='./logs',
   logging_steps=100,               # More frequent logging
   evaluation_strategy="epoch",
   save_strategy="epoch",
   load_best_model_at_end=True,
   push_to_hub=True,
   learning_rate=2e-5,             # Added explicit learning rate
   gradient_accumulation_steps=1    # No need for gradient accumulation with A100
)

def compute_metrics(pred):
   """
   Compute evaluation metrics for the model
   Args:
       pred: Contains predictions and label_ids
   Returns:
       dict: Dictionary of evaluation metrics
   """
   labels = pred.label_ids
   preds = pred.predictions.argmax(-1)
   precision, recall, f1, _ = precision_recall_fscore_support(
       labels,
       preds,
       average='binary',
       pos_label=1  # Explicitly set positive class
   )
   acc = accuracy_score(labels, preds)

   return {
       'accuracy': acc,
       'f1': f1,
       'precision': precision,
       'recall': recall
   }

# Initialize trainer
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=train_dataset,
   eval_dataset=test_dataset,
   compute_metrics=compute_metrics
)

# Print training configuration
print("Training Configuration:")
print(f"Model: bert-base-chinese")
print(f"Training samples: {len(train_dataset)}")
print(f"Test samples: {len(test_dataset)}")
print(f"Batch size: {training_args.per_device_train_batch_size}")
print(f"Number of epochs: {training_args.num_train_epochs}")

# Start training
trainer.train()

# Evaluate model
eval_results = trainer.evaluate()
print("\nEvaluation Results:", eval_results)

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of ü§ó Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Training Configuration:
Model: bert-base-chinese
Training samples: 95990
Test samples: 23998
Batch size: 32
Number of epochs: 3


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ¬∑¬∑¬∑¬∑¬∑¬∑¬∑¬∑¬∑¬∑


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


No files have been modified since last commit. Skipping to prevent empty commit.



Evaluation Results: {'eval_loss': 0.03648396208882332, 'eval_accuracy': 0.9840403366947246, 'eval_f1': 0.9839916405433646, 'eval_precision': 1.0, 'eval_recall': 0.9684877406615107, 'eval_runtime': 50.6448, 'eval_samples_per_second': 473.849, 'eval_steps_per_second': 7.405, 'epoch': 3.0}


In [ ]:
from transformers import pipeline
import torch

def test_sentiment(texts, yourmodel):
    """
    Test sentiment analysis model with given texts
    """
    # Create sentiment analyzer pipeline
    analyzer = pipeline(
        "sentiment-analysis",
        model=yourmodel,  # your model from HuggingFace Hub
        tokenizer="bert-base-chinese",
        device=0 if torch.cuda.is_available() else -1
    )

    # Process each text
    for text in texts:
        result = analyzer(text)[0]
        sentiment = "positive" if result['label'] == 'LABEL_1' else "negative"
        print(f"\nText: {text}")
        print(f"Sentiment: {sentiment}")
        print(f"Confidence: {result['score']:.4f}")

# Test with example texts
test_texts = [
    "ËøôÂÆ∂Â∫óÁöÑËèúÁúüÈ¶ôÔºå‰∏ãÊ¨°ËøòÊù•ÔºÅ",         # The food is delicious, will come again
    "Ë¥®ÈáèÊúâÈóÆÈ¢òÔºå‰∏çÊé®ËçêË¥≠‰π∞„ÄÇ",           # Quality issues, not recommended
    "Âø´ÈÄíÂæàÂø´ÔºåÂåÖË£ÖÂÆåÊï¥„ÄÇ",               # Fast delivery, good packaging
    "ÂïÜÂÆ∂ÊÄÅÂ∫¶Âæà‰∏çÂ•ΩÔºåÁîüÊ∞î„ÄÇ",             # Bad merchant attitude, angry
    "ÈùûÂ∏∏Êª°ÊÑèÔºåË∂ÖÂá∫È¢ÑÊúü„ÄÇ",               # Very satisfied, exceeded expectations
    "ÈöæÂêÉÂà∞ÊûÅÁÇπÔºåÂ§™Á≥üÁ≥ï‰∫Ü„ÄÇ",             # Extremely bad taste, terrible
    "Á©øÁùÄÂæàËàíÊúçÔºåÂ∞∫Á†ÅÂêàÈÄÇ„ÄÇ",             # Comfortable to wear, good size
    "ÂçñÂÆ∂ÊúçÂä°ÁâπÂà´Â•ΩÔºÅ",                   # Great service from seller
    "‰∏çÂÄºËøô‰∏™‰ª∑Èí±ÔºåÂêéÊÇî‰π∞‰∫Ü„ÄÇ",           # Not worth the price, regret buying
    "‰∫ßÂìÅÂÆåÂÖ®ÊòØÂûÉÂúæÔºåÊ∞îÊ≠ª‰∫Ü„ÄÇ"            # Product is totally garbage, very angry
]

# Run the test
test_sentiment(test_texts, "BarryzZ/sentiment-weibo-100k-fine-tuned-bert")

Device set to use cuda:0



Text: ËøôÂÆ∂Â∫óÁöÑËèúÁúüÈ¶ôÔºå‰∏ãÊ¨°ËøòÊù•ÔºÅ
Sentiment: positive
Confidence: 1.0000

Text: Ë¥®ÈáèÊúâÈóÆÈ¢òÔºå‰∏çÊé®ËçêË¥≠‰π∞„ÄÇ
Sentiment: positive
Confidence: 1.0000

Text: Âø´ÈÄíÂæàÂø´ÔºåÂåÖË£ÖÂÆåÊï¥„ÄÇ
Sentiment: positive
Confidence: 1.0000

Text: ÂïÜÂÆ∂ÊÄÅÂ∫¶Âæà‰∏çÂ•ΩÔºåÁîüÊ∞î„ÄÇ
Sentiment: positive
Confidence: 1.0000

Text: ÈùûÂ∏∏Êª°ÊÑèÔºåË∂ÖÂá∫È¢ÑÊúü„ÄÇ
Sentiment: positive
Confidence: 1.0000

Text: ÈöæÂêÉÂà∞ÊûÅÁÇπÔºåÂ§™Á≥üÁ≥ï‰∫Ü„ÄÇ
Sentiment: positive
Confidence: 1.0000

Text: Á©øÁùÄÂæàËàíÊúçÔºåÂ∞∫Á†ÅÂêàÈÄÇ„ÄÇ
Sentiment: positive
Confidence: 1.0000

Text: ÂçñÂÆ∂ÊúçÂä°ÁâπÂà´Â•ΩÔºÅ
Sentiment: positive
Confidence: 1.0000

Text: ‰∏çÂÄºËøô‰∏™‰ª∑Èí±ÔºåÂêéÊÇî‰π∞‰∫Ü„ÄÇ
Sentiment: positive
Confidence: 1.0000

Text: ‰∫ßÂìÅÂÆåÂÖ®ÊòØÂûÉÂúæÔºåÊ∞îÊ≠ª‰∫Ü„ÄÇ
Sentiment: positive
Confidence: 0.9997


In [ ]:
from transformers import BertTokenizer
import torch

def predict_sentiment(text, model, device='cuda'):
    """
    ÂØπËæìÂÖ•ÁöÑ‰∏≠ÊñáÊñáÊú¨ËøõË°åÊÉÖÊÑüÈ¢ÑÊµã

    Args:
        text (str): ËæìÂÖ•ÁöÑ‰∏≠ÊñáÊñáÊú¨
        model: ËÆ≠ÁªÉÂ•ΩÁöÑÊ®°Âûã
        device (str): ËøêË°åËÆæÂ§á ('cuda' Êàñ 'cpu')

    Returns:
        dict: ÂåÖÂê´È¢ÑÊµãÁªìÊûúÂíåÊ¶ÇÁéáÁöÑÂ≠óÂÖ∏
    """
    # Âä†ËΩΩÂàÜËØçÂô®
    tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

    # È¢ÑÂ§ÑÁêÜÊñáÊú¨
    encoded = tokenizer(
        text,
        truncation=True,
        padding=True,
        max_length=128,
        return_tensors='pt'
    )

    # Â∞ÜËæìÂÖ•ÁßªÂà∞ÊåáÂÆöËÆæÂ§á
    input_ids = encoded['input_ids'].to(device)
    attention_mask = encoded['attention_mask'].to(device)

    # ËÆæÁΩÆÊ®°Âûã‰∏∫ËØÑ‰º∞Ê®°Âºè
    model.eval()

    # ËøõË°åÈ¢ÑÊµã
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        probabilities = torch.softmax(outputs.logits, dim=1)
        prediction = torch.argmax(probabilities, dim=1)
        confidence = probabilities[0][prediction[0]].item()

    # ÊûÑÂª∫ÁªìÊûú
    sentiment = "Ê≠£Èù¢" if prediction.item() == 1 else "Ë¥üÈù¢"

    return {
        "text": text,
        "sentiment": sentiment,
        "confidence": f"{confidence:.2%}",
        "probabilities": {
            "Ë¥üÈù¢": f"{probabilities[0][0].item():.2%}",
            "Ê≠£Èù¢": f"{probabilities[0][1].item():.2%}"
        }
    }

# ÊµãËØïÊ†∑‰æã
def test_model(model, device='cuda'):
    """
    ‰ΩøÁî®‰∏Ä‰∫õÁ§∫‰æãÊñáÊú¨ÊµãËØïÊ®°Âûã

    Args:
        model: ËÆ≠ÁªÉÂ•ΩÁöÑÊ®°Âûã
        device (str): ËøêË°åËÆæÂ§á
    """
    test_texts = [
        "ËøôÈÉ®ÁîµÂΩ±ÁúüÁöÑÂæàÊ£íÔºåÊºîÂëòÊºîÊäÄÈÉΩÂæàÂ•ΩÔºÅ",
        "ÊúçÂä°ÊÄÅÂ∫¶Â∑ÆÔºåÂÆåÂÖ®‰∏çÊé®Ëçê„ÄÇ",
        "‰ª∑Ê†ºÂêàÁêÜÔºåË¥®Èáè‰πü‰∏çÈîô„ÄÇ",
        "Â§±ÊúõÔºåÊµ™Ë¥πÊó∂Èó¥ÂíåÈáëÈí±„ÄÇ",
        "‰∏ÄËà¨Ëà¨ÂêßÔºåÊ≤°ÊúâÁâπÂà´Â•Ω‰πüÊ≤°ÊúâÁâπÂà´Â∑Æ„ÄÇ"
    ]

    print("Ê®°ÂûãÊµãËØïÁªìÊûúÔºö")
    print("-" * 50)

    for text in test_texts:
        result = predict_sentiment(text, model, device)
        print(f"\nËæìÂÖ•ÊñáÊú¨Ôºö{result['text']}")
        print(f"È¢ÑÊµãÊÉÖÊÑüÔºö{result['sentiment']}")
        print(f"ÁΩÆ‰ø°Â∫¶Ôºö{result['confidence']}")
        print(f"Ê¶ÇÁéáÂàÜÂ∏ÉÔºö{result['probabilities']}")
        print("-" * 50)

In [ ]:
from transformers import pipeline, BertTokenizer

# Âõ†‰∏∫‰Ω†ÁöÑÊ®°ÂûãÊòØÂü∫‰∫ébert-base-chineseËÆ≠ÁªÉÁöÑ,ÊâÄ‰ª•‰ΩøÁî®ÂêåÊ†∑ÁöÑtokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

classifier = pipeline(
    "text-classification",
    model="BarryzZ/GISchat-weibo-100k-fine-tuned-bert",
    tokenizer=tokenizer
)

# ÊµãËØïÊñáÊú¨
text = "Â§™Á≥üÁ≥ï‰∫Ü"
result = classifier(text)
print(result)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'LABEL_1', 'score': 0.9629785418510437}]


In [ ]:
from transformers import BertTokenizer
import torch

def predict_sentiment(text, model, device='cuda'):
    """
    Predict sentiment for input Chinese text

    Args:
        text (str): Input Chinese text
        model: Trained model
        device (str): Computing device ('cuda' or 'cpu')

    Returns:
        dict: Dictionary containing prediction results and probabilities
    """
    # Load tokenizer
    tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

    # Preprocess text
    encoded = tokenizer(
        text,
        truncation=True,
        padding=True,
        max_length=128,
        return_tensors='pt'
    )

    # Move inputs to specified device
    input_ids = encoded['input_ids'].to(device)
    attention_mask = encoded['attention_mask'].to(device)

    # Set model to evaluation mode
    model.eval()

    # Make prediction
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        probabilities = torch.softmax(outputs.logits, dim=1)
        prediction = torch.argmax(probabilities, dim=1)
        confidence = probabilities[0][prediction[0]].item()

    # Build result dictionary
    sentiment = "Ê≠£Èù¢" if prediction.item() == 1 else "Ë¥üÈù¢"

    return {
        "text": text,
        "sentiment": sentiment,
        "confidence": f"{confidence:.2%}",
        "probabilities": {
            "negative": f"{probabilities[0][0].item():.2%}",
            "positive": f"{probabilities[0][1].item():.2%}"
        }
    }

# Test examples
def test_model(model, device='cuda'):
    """
    Test the model with example texts

    Args:
        model: Trained model
        device (str): Computing device
    """
    test_texts = [
        "ËøôÈÉ®ÁîµÂΩ±ÁúüÁöÑÂæàÊ£íÔºåÊºîÂëòÊºîÊäÄÈÉΩÂæàÂ•ΩÔºÅ",  # This movie is great, the actors performed well!
        "ÊúçÂä°ÊÄÅÂ∫¶Â∑Æ„ÄÇ",  # Poor service, definitely not recommended.
        "‰ª∑Ê†ºÂêàÁêÜÔºåË¥®Èáè‰πü‰∏çÈîô„ÄÇ",  # Reasonable price and good quality.
        "Â§±ÊúõÔºåÊµ™Ë¥πÊó∂Èó¥ÂíåÈáëÈí±„ÄÇ",  # Disappointed, waste of time and money.
        "‰∏çÂèØ‰ª•ÔºåËøô‰∏™ÁâπÂà´Â∑Æ„ÄÇ"  # Just average, neither particularly good nor bad.
    ]

    print("Model Test Results:")
    print("-" * 50)

    for text in test_texts:
        result = predict_sentiment(text, model, device)
        print(f"\nInput text: {result['text']}")
        print(f"Predicted sentiment: {result['sentiment']}")
        print(f"Confidence: {result['confidence']}")
        print(f"Probability distribution: {result['probabilities']}")
        print("-" * 50)

In [ ]:
# Test single text
text = "ËøôÂÆ∂È§êÂéÖÁöÑËèúÂìÅÈùûÂ∏∏ÁæéÂë≥ÔºåÊúçÂä°‰πüÂæàË¥¥ÂøÉÔºÅ"  # The food at this restaurant is delicious and the service is thoughtful!
result = predict_sentiment(text, model)
print(result)

# Or run the example test set
test_model(model)

{'text': 'ËøôÂÆ∂È§êÂéÖÁöÑËèúÂìÅÈùûÂ∏∏ÁæéÂë≥ÔºåÊúçÂä°‰πüÂæàË¥¥ÂøÉÔºÅ', 'sentiment': 'Ê≠£Èù¢', 'confidence': '100.00%', 'probabilities': {'negative': '0.00%', 'positive': '100.00%'}}
Model Test Results:
--------------------------------------------------

Input text: ËøôÈÉ®ÁîµÂΩ±ÁúüÁöÑÂæàÊ£íÔºåÊºîÂëòÊºîÊäÄÈÉΩÂæàÂ•ΩÔºÅ
Predicted sentiment: Ê≠£Èù¢
Confidence: 100.00%
Probability distribution: {'negative': '0.00%', 'positive': '100.00%'}
--------------------------------------------------

Input text: ÊúçÂä°ÊÄÅÂ∫¶Â∑Æ„ÄÇ
Predicted sentiment: Ê≠£Èù¢
Confidence: 99.99%
Probability distribution: {'negative': '0.01%', 'positive': '99.99%'}
--------------------------------------------------

Input text: ‰ª∑Ê†ºÂêàÁêÜÔºåË¥®Èáè‰πü‰∏çÈîô„ÄÇ
Predicted sentiment: Ê≠£Èù¢
Confidence: 100.00%
Probability distribution: {'negative': '0.00%', 'positive': '100.00%'}
--------------------------------------------------

Input text: Â§±ÊúõÔºåÊµ™Ë¥πÊó∂Èó¥ÂíåÈáëÈí±„ÄÇ
Predicted sentiment: Ë¥üÈù¢
Confidence: 94.48%
Pr

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # ÂàÜÂà´ËÆ°ÁÆóÊØè‰∏™Á±ªÂà´ÁöÑÊåáÊ†á
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels,
        preds,
        average=None,  # ÂàÜÂà´ËÆ°ÁÆóÊØè‰∏™Á±ªÂà´ÁöÑÊåáÊ†á
        labels=[0, 1]  # ÊòéÁ°ÆÊåáÂÆöÊ†áÁ≠æÈ°∫Â∫è
    )
    acc = accuracy_score(labels, preds)

    # ËÆ°ÁÆóÊ∑∑Ê∑ÜÁü©Èòµ
    conf_mat = confusion_matrix(labels, preds)

    return {
        'accuracy': acc,
        'f1_neg': f1[0],  # Ê∂àÊûÅÁ±ªÁöÑF1
        'f1_pos': f1[1],  # ÁßØÊûÅÁ±ªÁöÑF1
        'f1_avg': f1.mean(),
        'precision_neg': precision[0],
        'precision_pos': precision[1],
        'recall_neg': recall[0],
        'recall_pos': recall[1],
        'confusion_matrix': conf_mat.tolist()
    }


# ÊµãËØï‰øÆÊîπÂêéÁöÑÂáΩÊï∞
test_texts = [
    "ËøôÈÉ®ÁîµÂΩ±ÁúüÁöÑÂæàÊ£íÔºåÊºîÂëòÊºîÊäÄÈÉΩÂæàÂ•ΩÔºÅ",
    "ÊúçÂä°ÊÄÅÂ∫¶Â∑ÆÔºå‰ª∑Ê†ºËøòË¥µÔºå‰∏ç‰ºöÂÜçÊù•‰∫Ü",
    "‰∏ÄËà¨Ëà¨ÂêßÔºåÊ≤°‰ªÄ‰πàÁâπÂà´ÁöÑ",
    "Ë∂ÖÁ∫ßÂñúÊ¨¢ËøôÂÆ∂Â∫óÔºå‰∏úË•ø‰æøÂÆúËøòÂ•ΩÂêÉ"
]

for text in test_texts:
    result = predict_sentiment(text)
    print(f"\nÊñáÊú¨: {text}")
    print(f"ÊÉÖÊÑü: {result[0]['sentiment']}")
    print(f"ÁΩÆ‰ø°Â∫¶: {result[0]['score']:.4f}")


ÊñáÊú¨: ËøôÈÉ®ÁîµÂΩ±ÁúüÁöÑÂæàÊ£íÔºåÊºîÂëòÊºîÊäÄÈÉΩÂæàÂ•ΩÔºÅ
ÊÉÖÊÑü: Ê∂àÊûÅ
ÁΩÆ‰ø°Â∫¶: 0.9648

ÊñáÊú¨: ÊúçÂä°ÊÄÅÂ∫¶Â∑ÆÔºå‰ª∑Ê†ºËøòË¥µÔºå‰∏ç‰ºöÂÜçÊù•‰∫Ü
ÊÉÖÊÑü: Ê∂àÊûÅ
ÁΩÆ‰ø°Â∫¶: 0.9653

ÊñáÊú¨: ‰∏ÄËà¨Ëà¨ÂêßÔºåÊ≤°‰ªÄ‰πàÁâπÂà´ÁöÑ
ÊÉÖÊÑü: Ê∂àÊûÅ
ÁΩÆ‰ø°Â∫¶: 0.9789

ÊñáÊú¨: Ë∂ÖÁ∫ßÂñúÊ¨¢ËøôÂÆ∂Â∫óÔºå‰∏úË•ø‰æøÂÆúËøòÂ•ΩÂêÉ
ÊÉÖÊÑü: Ê∂àÊûÅ
ÁΩÆ‰ø°Â∫¶: 0.9879


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import pandas as pd
from datasets import load_dataset
from transformers import EarlyStoppingCallback
from huggingface_hub import login

# Login to Hugging Face Hub
login("your-token")

# Load dataset
ds = load_dataset("dirtycomputer/weibo_senti_100k")
df = pd.DataFrame(ds['train'])

# Split dataset
train_df, test_df = train_test_split(df, test_size=0.2, shuffle=True, random_state=42)

# Check GPU availability
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("Using CPU")

# Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

# Custom Dataset class
class TextDataset(Dataset):
    def __init__(self, tokenizer, texts, labels):
        self.encodings = tokenizer(
            texts,
            truncation=True,
            padding=True,
            max_length=512,
            return_tensors='pt'
        )
        self.labels = torch.tensor(labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'labels': self.labels[idx]
        }

    def __len__(self):
        return len(self.labels)

# Explicit label mapping
label_to_index = {
    0: 0,  # Negative
    1: 1   # Positive
}

# Create datasets
train_labels = [label_to_index[label] for label in train_df['label']]
test_labels = [label_to_index[label] for label in test_df['label']]

# Create datasets with verified labels
train_dataset = TextDataset(tokenizer, train_df['review'].tolist(), train_labels)
test_dataset = TextDataset(tokenizer, test_df['review'].tolist(), test_labels)

# Verify final mapping
print("\nFinal verification:")
print("Training set label distribution:", pd.Series(train_labels).value_counts())
print("Test set label distribution:", pd.Series(test_labels).value_counts())

# Initialize model
model = BertForSequenceClassification.from_pretrained(
    'bert-base-chinese',
    num_labels=2
)
model = model.to(device)

# ‰ºòÂåñÂêéÁöÑËÆ≠ÁªÉÂèÇÊï∞
training_args = TrainingArguments(
    output_dir='sentiment-weibo-100k-fine-tuned-bert-test',
    num_train_epochs=5,
    per_device_train_batch_size=64,    # Â¢ûÂä†Âà∞64 (ÂéüÊù•ÊòØ16)
    per_device_eval_batch_size=128,    # Â¢ûÂä†Âà∞128 (ÂéüÊù•ÊòØ32)
    learning_rate=2e-5,                # Áï•ÂæÆÊèêÈ´òÂ≠¶‰π†Áéá
    warmup_ratio=0.1,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    evaluation_strategy="steps",
    eval_steps=200,                    # ÂáèÂ∞ëËØÑ‰º∞Èó¥Èöî (ÂéüÊù•ÊòØ500)
    save_strategy="steps",
    save_steps=200,                    # ÂáèÂ∞ë‰øùÂ≠òÈó¥Èöî
    load_best_model_at_end=True,
    metric_for_best_model="f1_avg",
    push_to_hub=True,
    gradient_accumulation_steps=1,     # Á°Æ‰øù‰∏ç‰ΩøÁî®Ê¢ØÂ∫¶Á¥ØÁßØ
    fp16=True                          # ÂêØÁî®Ê∑∑ÂêàÁ≤æÂ∫¶ËÆ≠ÁªÉ
)
# Compute metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    precision, recall, f1, _ = precision_recall_fscore_support(
        labels,
        preds,
        average=None,
        labels=[0, 1]
    )
    acc = accuracy_score(labels, preds)
    conf_mat = confusion_matrix(labels, preds)

    return {
        'accuracy': acc,
        'f1_neg': f1[0],
        'f1_pos': f1[1],
        'f1_avg': f1.mean(),
        'precision_neg': precision[0],
        'precision_pos': precision[1],
        'recall_neg': recall[0],
        'recall_pos': recall[1],
        'confusion_matrix': conf_mat.tolist()
    }

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# Print dataset statistics before training
print("\nDataset Statistics:")
print(f"Training samples: {len(train_dataset)}")
print(f"Test samples: {len(test_dataset)}")
print("\nLabel Distribution:")
print("Training:", pd.Series([d['labels'].item() for d in train_dataset]).value_counts())
print("Testing:", pd.Series([d['labels'].item() for d in test_dataset]).value_counts())

# Start training
trainer.train()

# Evaluate model
eval_results = trainer.evaluate()
print("\nFinal Evaluation Results:")
for metric, value in eval_results.items():
    if isinstance(value, float):
        print(f"{metric}: {value:.4f}")
    else:
        print(f"{metric}: {value}")

Using GPU: NVIDIA A100-SXM4-40GB

Final verification:
Training set label distribution: 0    48151
1    47839
Name: count, dtype: int64
Test set label distribution: 1    12154
0    11844
Name: count, dtype: int64


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of ü§ó Transformers. Use `eval_strategy` instead
  warnings.warn(



Dataset Statistics:
Training samples: 95990
Test samples: 23998

Label Distribution:
Training: 0    48151
1    47839
Name: count, dtype: int64
Testing: 1    12154
0    11844
Name: count, dtype: int64


Trainer is attempting to log a value of "[[11808, 36], [380, 11774]]" of type <class 'list'> for key "eval/confusion_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[[11844, 0], [398, 11756]]" of type <class 'list'> for key "eval/confusion_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[[11844, 0], [386, 11768]]" of type <class 'list'> for key "eval/confusion_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[[11844, 0], [385, 11769]]" of type <class 'list'> for key "eval/confusion_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[[11844, 0], [384, 11770]]

Trainer is attempting to log a value of "[[11844, 0], [384, 11770]]" of type <class 'list'> for key "eval/confusion_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.



Final Evaluation Results:
eval_loss: 0.0438
eval_accuracy: 0.9840
eval_f1_neg: 0.9840
eval_f1_pos: 0.9839
eval_f1_avg: 0.9840
eval_precision_neg: 0.9686
eval_precision_pos: 1.0000
eval_recall_neg: 1.0000
eval_recall_pos: 0.9684
eval_confusion_matrix: [[11844, 0], [384, 11770]]
eval_runtime: 8.5367
eval_samples_per_second: 2811.1650
eval_steps_per_second: 22.0230
epoch: 1.0667


In [ ]:
# Test with example texts
test_texts = [
    "ËøôÂÆ∂Â∫óÁöÑËèúÁúüÈ¶ôÔºå‰∏ãÊ¨°ËøòÊù•ÔºÅ",         # The food is delicious, will come again
    "Ë¥®ÈáèÊúâÈóÆÈ¢òÔºå‰∏çÊé®Ëçê„ÄÇ",           # Quality issues, not recommended
    "Âø´ÈÄíÂæàÂø´ÔºåÂåÖË£ÖÂÆåÊï¥„ÄÇ",               # Fast delivery, good packaging
    "ÂïÜÂÆ∂ÊÄÅÂ∫¶‰∏çÂ•ΩÔºåÁîüÊ∞î„ÄÇ",             # Bad merchant attitude, angry
    "ÈùûÂ∏∏Êª°ÊÑèÔºåË∂ÖÂá∫È¢ÑÊúü„ÄÇ",               # Very satisfied, exceeded expectations
    "ÈöæÂêÉÂà∞ÊûÅÁÇπÔºåÂ§™Á≥üÁ≥ï‰∫Ü„ÄÇ",             # Extremely bad taste, terrible
    "Á©øÁùÄÂæàËàíÊúçÔºåÂ∞∫Á†ÅÂêàÈÄÇ„ÄÇ",             # Comfortable to wear, good size
    "ÂçñÂÆ∂ÊúçÂä°ÁâπÂà´Â•ΩÔºÅ",                   # Great service from seller
    "‰∏çÂÄºËøô‰∏™‰ª∑Èí±ÔºåÂêéÊÇî‰π∞‰∫Ü„ÄÇ",           # Not worth the price, regret buying
    "‰∫ßÂìÅÂÆåÂÖ®ÊòØÂûÉÂúæÔºåÊ∞îÊ≠ª‰∫Ü„ÄÇ"            # Product is totally garbage, very angry
]

# Run the test
test_sentiment(test_texts, "BarryzZ/sentiment-weibo-100k-fine-tuned-bert-test")

Device set to use cuda:0



Text: ËøôÂÆ∂Â∫óÁöÑËèúÁúüÈ¶ôÔºå‰∏ãÊ¨°ËøòÊù•ÔºÅ
Sentiment: positive
Confidence: 0.9923

Text: Ë¥®ÈáèÊúâÈóÆÈ¢òÔºå‰∏çÊé®Ëçê„ÄÇ
Sentiment: negative
Confidence: 0.8533

Text: Âø´ÈÄíÂæàÂø´ÔºåÂåÖË£ÖÂÆåÊï¥„ÄÇ
Sentiment: positive
Confidence: 0.9878

Text: ÂïÜÂÆ∂ÊÄÅÂ∫¶‰∏çÂ•ΩÔºåÁîüÊ∞î„ÄÇ
Sentiment: negative
Confidence: 0.9732

Text: ÈùûÂ∏∏Êª°ÊÑèÔºåË∂ÖÂá∫È¢ÑÊúü„ÄÇ
Sentiment: positive
Confidence: 0.9791

Text: ÈöæÂêÉÂà∞ÊûÅÁÇπÔºåÂ§™Á≥üÁ≥ï‰∫Ü„ÄÇ
Sentiment: negative
Confidence: 0.8653

Text: Á©øÁùÄÂæàËàíÊúçÔºåÂ∞∫Á†ÅÂêàÈÄÇ„ÄÇ
Sentiment: positive
Confidence: 0.9907

Text: ÂçñÂÆ∂ÊúçÂä°ÁâπÂà´Â•ΩÔºÅ
Sentiment: positive
Confidence: 0.9922

Text: ‰∏çÂÄºËøô‰∏™‰ª∑Èí±ÔºåÂêéÊÇî‰π∞‰∫Ü„ÄÇ
Sentiment: negative
Confidence: 0.8147

Text: ‰∫ßÂìÅÂÆåÂÖ®ÊòØÂûÉÂúæÔºåÊ∞îÊ≠ª‰∫Ü„ÄÇ
Sentiment: negative
Confidence: 0.9863


In [ ]:
test_texts = [
    # Âº∫ÁÉàÊ≠£Èù¢
    "ÊàëËÄÉ‰∏äÁ†îÁ©∂Áîü‰∫ÜÔºÅ",
    "‰ªäÂ§©‰ªñÂêëÊàëÊ±ÇÂ©ö‰∫ÜÔºÅ",
    "Áªà‰∫é‰π∞Âà∞Ê¢¶ÊÉ≥ÁöÑÊàøÂ≠ê",
    "‰∏≠‰∫Ü‰∫îÁôæ‰∏áÂ§ßÂ•ñÔºÅ",

    # Âº∫ÁÉàË¥üÈù¢
    "Ë¢´Ë£ÅÂëò‰∫ÜÔºåÂ•ΩÁªùÊúõ",
    "‰ø°‰ªªÁöÑ‰∫∫ËÉåÂèõÊàë",
    "ÈáçË¶ÅÁöÑÊñá‰ª∂ÂÖ®‰∏¢‰∫Ü",
    "ÂèàË¢´Êâ£Â∑•ËµÑ‰∫ÜÔºåÊ∞îÊ≠ª",

    # ÊÑ§ÊÄí
    "ÂÅ∑ÊàëÁöÑËΩ¶ÔºåÊ∑∑ËõãÔºÅ",
    "È™óÂ≠êÂÖ¨Âè∏ÔºåÊàëË¶ÅÊä•Ë≠¶",
    "ÂçäÂ§úË£Ö‰øÆÔºåÁÉ¶Ê≠ª‰∫Ü",
    "ÂïÜÂÆ∂Â§™Âùë‰∫∫‰∫ÜÔºÅ",

    # ÊÉäÂñú
    "ÂÆùÂÆù‰ºöËµ∞Ë∑Ø‰∫ÜÔºÅ",
    "ÂçáËÅåÂä†Ëñ™Âï¶ÔºÅ",
    "ËÆ∫ÊñáÂèëË°®ÊàêÂäüÔºÅ",
    "Êî∂Âà∞offer‰∫ÜÔºÅ"
]
# Run the test
test_sentiment(test_texts, "BarryzZ/sentiment-weibo-100k-fine-tuned-bert-test")

Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset



Text: ÊàëËÄÉ‰∏äÁ†îÁ©∂Áîü‰∫ÜÔºÅ
Sentiment: positive
Confidence: 0.8713

Text: ‰ªäÂ§©‰ªñÂêëÊàëÊ±ÇÂ©ö‰∫ÜÔºÅ
Sentiment: positive
Confidence: 0.6087

Text: Áªà‰∫é‰π∞Âà∞Ê¢¶ÊÉ≥ÁöÑÊàøÂ≠ê
Sentiment: positive
Confidence: 0.7931

Text: ‰∏≠‰∫Ü‰∫îÁôæ‰∏áÂ§ßÂ•ñÔºÅ
Sentiment: positive
Confidence: 0.6070

Text: Ë¢´Ë£ÅÂëò‰∫ÜÔºåÂ•ΩÁªùÊúõ
Sentiment: negative
Confidence: 0.9973

Text: ‰ø°‰ªªÁöÑ‰∫∫ËÉåÂèõÊàë
Sentiment: negative
Confidence: 0.9572

Text: ÈáçË¶ÅÁöÑÊñá‰ª∂ÂÖ®‰∏¢‰∫Ü
Sentiment: negative
Confidence: 0.9941

Text: ÂèàË¢´Êâ£Â∑•ËµÑ‰∫ÜÔºåÊ∞îÊ≠ª
Sentiment: negative
Confidence: 0.9963

Text: ÂÅ∑ÊàëÁöÑËΩ¶ÔºåÊ∑∑ËõãÔºÅ
Sentiment: negative
Confidence: 0.9664

Text: È™óÂ≠êÂÖ¨Âè∏ÔºåÊàëË¶ÅÊä•Ë≠¶
Sentiment: negative
Confidence: 0.9750

Text: ÂçäÂ§úË£Ö‰øÆÔºåÁÉ¶Ê≠ª‰∫Ü
Sentiment: negative
Confidence: 0.9906

Text: ÂïÜÂÆ∂Â§™Âùë‰∫∫‰∫ÜÔºÅ
Sentiment: negative
Confidence: 0.8367

Text: ÂÆùÂÆù‰ºöËµ∞Ë∑Ø‰∫ÜÔºÅ
Sentiment: positive
Confidence: 0.9125

Text: ÂçáËÅåÂä†Ëñ™Âï¶ÔºÅ
Sentiment: positive
Confidence: 0.9727

Text: ËÆ

In [ ]:
# Êü•ÁúãÊµãËØïÈõÜ‰∏≠ÁöÑÊ∂àÊûÅÊ†∑Êú¨
print("Ê∂àÊûÅÊ†∑Êú¨Á§∫‰æãÔºö")
for i, (text, label) in enumerate(zip(test_df['review'], test_df['label'])):
   if label == 0:  # 0Ë°®Á§∫Ê∂àÊûÅ
       print(f"\n{i+1}. {text}")
       if i >= 29:  # ÊòæÁ§∫Ââç10‰∏™Ê†∑Êú¨
           break

# ÁªüËÆ°‰∏Ä‰∫õÂü∫Êú¨‰ø°ÊÅØ
neg_samples = test_df[test_df['label'] == 0]
print(f"\nÊ∂àÊûÅÊ†∑Êú¨ÊÄªÊï∞: {len(neg_samples)}")
print("\nÊ∂àÊûÅÊ†∑Êú¨Â≠óÊï∞ÂàÜÂ∏ÉÔºö")
print(neg_samples['review'].str.len().describe())

Ê∂àÊûÅÊ†∑Êú¨Á§∫‰æãÔºö

4. ÁîµËßÜÈáåÂú®ÂÖ¨Â∏ÉÈÄöÁºâÁäØÁöÑÁîªÂÉèÔºåÂßêÊÄªÊÑüËßâÂì™Èáå‰∏çÂØπÂïä[Êôï]

5. ÊàëÊÉ≥Âá∫ÂéªÁé©ÔºåÂ•ΩÁÉ¶ÂïäÔºåÂ§©Â§©ÈÉΩÂú®Ëøô‰∏™Ê•ºÈáå[ÊÄí]‰ΩïÊó•ËÉΩËßÅÂÖâÊòé[ÊÇ≤‰º§]

6. Â•ΩËàíÊúç„ÄÇ„ÄÇ„ÄÇÊúç‰∫Ü„ÄÇ„ÄÇ„ÄÇ//@Áå¥Â≠êÊê¨ÊïëÂÖµ‰πüÊïë‰∏ç‰∫Ü‰Ω†:ÊúÄÂêéÈÇ£‰ΩçÈ™öÂπ¥‰Ω†Ëøô‰πàÊ∑°ÂÆöÁúüÁöÑÂ•ΩÂêóÂìàÂìàÂìàÂìàÔºÅ//@Á©øÁùÄÈ©¨Áî≤ÈóØÊ±üÊπñ: [‰º§ÂøÉ] Â≠¶Â¶πÊòØÂ≠¶ÈïøÁöÑ„ÄÇ„ÄÇ„ÄÇÂ≠¶ÂºüÂøÖË¶ÅÁöÑÊó∂ÂÄô‰πüÊòØÂ≠¶ÈïøÁöÑ„ÄÇ //@Ëá≠ÊµÅÊ∞ìÁà±ÈÄõË°ó: 3ÂàÜ18Áßí‚Ä¶‚Ä¶Á¨ëÂ∞ø‰∫ÜÂ•Ω‰πà‚Ä¶‚Ä¶

7. ÂØπ@kunÂ∞èË•ø ËØ¥ÔºöË¢´‰Ω†ËôêÂà∞‰∫Ü „ÄêÈÇ£Âπ¥ÔºåÂº†Èõ®ÁîüËøòÊ≤°Ê≠ªÔºåÁéãÊù∞Ê≠£Á∫¢‚Üêkill me softly with his song„Äë [Ê≥™]

8. ‰ªäÂ§©ÁâπÂà´Á¥Ø[ÊÇ≤‰º§]

10. ÂõûÂ§ç@ÈóªÈóªÊòØÈ¶ôÁöÑËøòÊòØËá≠DIÂïä:Ê¨¢Ëøé‰∏ãÊ¨°ÂÜçÊù•ÔºÅ@ÈáëÈôµËøûÈîÅÈÖíÂ∫ó //@ÈóªÈóªÊòØÈ¶ôÁöÑËøòÊòØËá≠DIÂïä:ÊÄªÁªüÂ•óÊàøÁúüÁöÑÂæàÈ´òÁ´ØÂ§ßÊ∞îÂì¶ÔºåÈÅóÊÜæÊ≤°ÁúãÂà∞Ë•øÂºèÊÄªÂ•ó‰ªÄ‰πàÊ†∑[Ê≥™] //@ÂÆÅÊù∞Luck_Zhu:@Áü•ÊÄßÂÆáÁ∫Ø @‰∏öÊ¢ÖCandy //@Âú£Âç°Ê¥õÁê≥: ÁúüÂøÉÁªôÂäõÁöÑÈ´òÊ∏ÖÂõæÔºÅÊëÑÂΩ±ÊäÄÊúØ‰∫ÜÂæóÔºÅ //@ËãèÂ∑ûÈáëÈôµËßÇÂõ≠ÂõΩÈôÖÈÖíÂ∫ó: È´òÊ∏ÖÁÖßÁâáÊûúÁÑ∂ÁªôÂ